In [ ]:
from __future__ import print_function

In [ ]:
# optionally copy doab.json from DOAB repo

import shutil

# toggle the boolean to set whether to copy
if (False):
    shutil.copyfile("/Users/raymondyee/D/Document/Gluejar/Gluejar.github/DOAB/doab.json",
                    "../bookdata/doab.json")

# load up django settings


In [ ]:
import os
import django

# http://stackoverflow.com/questions/24793351/django-appregistrynotready

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'regluit.settings.me')
django.setup()

# Loading the list of books

In [ ]:
import json
import codecs
s = codecs.open("../bookdata/doab.json", encoding='UTF-8').read()
records = json.loads(s)
records[1]

I need to remind myself of how to check that there are no outstanding celery jobs after I do this loading. 

I have a technique for using `django-celery` monitoring that works on redis (what we use on just and production) -- but not laptop (http://stackoverflow.com/a/5451479/7782).  I think a workable way is to look at the celery_taskmeta table.

In [ ]:
limit = None

In [ ]:
from regluit.core.loaders import doab

file_path = '../bookdata/doab.json'

doab.load_doab_records(file_path, limit=int(limit))

In [ ]:
import djcelery
[t.status for t in djcelery.models.TaskMeta.objects.all()]

# Tests for the loading

   * can we find all the the URLs?
   * is it associated with the the right doab_id?
   * all the ISBNs loaded?
   * which books are not matched with Google Books IDs -- and therefore might require URLs for covers?
   * did I make sure the edition I'm attaching the ebooks to is the "selected edition"?
   * for editions that I create [and maybe all editions?], attach a cover_image from DOAB.
   * all clustered around the same work?  (or do I need to do further merging?)
   * are we creating extraneous works?
   * subject metadata
   * are we loading all the useful metadata? 
   * is the loading script idempotent?


## important limit to testing

I have written code to handle the loading of all associated ISBNs with DOAB records -- but we upload only records with non-null licenses, we will have only one ISBN per DOAB record for records with known licenses. So the loading of works for which we know the license won't exercise the code in question:
https://github.com/Gluejar/regluit/blob/5b3a8d7b1302bc1b1985c675add06c345567a7a1/core/doab.py#L91
I also checked that there is no intersection of DOAB ids betwen records with known licenses and those that don't.

In [ ]:
from regluit.core.models import Work, Edition, Ebook, Identifier
from regluit.core.isbn import ISBN
from itertools import islice

import traceback
import sys


tests_exceptions = []
no_google_book_id  = []
all_problems = []
cover_problems = []

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'regluit.settings.me')
django.setup()

records_to_load = list(islice(records,limit))

for record in islice(records_to_load, None):
    d = dict(record)
    
    problems = []
    
    try:
        
        # has a work been associated with the doab_id?

        work = Identifier.objects.get(type='doab', 
                value=d.get('doab_id')).work

        edition = work.selected_edition
        
        # check only one ebook with this URL.
        
        # check for url if format
        if d.get('format') in ('pdf', 'epub', 'mobi'):
            ebooks = Ebook.objects.filter(url=d.get('url'))
            if len(ebooks) != 1:
                problems.append("len(ebooks): {}".format(len(ebooks)))
            
        # all the ISBNs loaded?
        # this code might be a bit inefficient given there might only be one isbn per record
        
        isbns = [ISBN(i).to_string() for i in d.get('isbns')]
        if not(set(isbns) == set([id_.value for id_ in Identifier.objects.filter(type="isbn", 
                                                                   value__in=isbns)])):
            problems.append("isbns not matching")
            
        if problems:
            all_problems.append((d, problems))
            
        # check on presence of Google books id
        if len(edition.identifiers.filter(type="goog")) < 1:
            no_google_book_id.append(d)

        # check on the cover URLs
        #print (edition.work.cover_image_small())
        if edition.work.cover_image_small().find("amazonaws") < 0:
            cover_problems.append((d))
    
    except Exception as e:
        (exc_type, exc_value, exc_tb) = sys.exc_info()
        stack_trace = " ".join(traceback.format_exception(exc_type, exc_value, exc_tb))

        tests_exceptions.append((d, (e, stack_trace)))
  
print ("number of records loaded",  len(records_to_load))
print ()
print ("all_problems", all_problems)
print ()
print ("tests_exceptions", tests_exceptions)
print ()
print ("no_google_book_id", no_google_book_id)
print ()
print ("cover problems", cover_problems)

In [ ]:
all_problems[0]

In [ ]:
# work through test exceptions

for (d, (e, trace)) in tests_exceptions:
    print(d, trace)

In [ ]:
# work through specific example

d = dict(records_to_load[2])

url = d.get('url')
print(url)

# has a work been associated with the doab_id?

work = Identifier.objects.get(type='doab', 
        value=d.get('doab_id')).work

edition = work.selected_edition

# check for url if format
if d.get('format') in ('pdf', 'epub', 'mobi'):
    ebooks = Ebook.objects.filter(url=url)
    print (len(ebooks))

# google id
print ( len(edition.identifiers.filter(type="goog")), len(edition.work.identifiers.filter(type="goog")))

In [ ]:
d

In [ ]:
ebook = doab.load_doab_edition(**d)
ebook

In [ ]:
ebook is None

In [ ]:
d

In [ ]:
from regluit.core import bookloader 
isbn = '9788575414088'

ed1 = bookloader.add_by_isbn(isbn)
ed1

In [ ]:
all_problems

# Stop

In [ ]:
raise Exception("Stop here")

In [ ]:
# invalid ISBNs?

for (d, p) in all_problems:
    print (d['isbns'][0], ISBN(d['isbns'][0]).valid)

In [ ]:
[(d['doab_id'], d['isbns'][0]) for d in no_google_book_id]

In [ ]:
# it is possible to do a query for a whole set of values, a technique I might make use of.
# http://stackoverflow.com/a/9304968
# e.g., Blog.objects.filter(pk__in=[1,4,7])

urls = [dict(record).get('url') for record in records_to_load]
set([ebook.url for ebook in Ebook.objects.filter(url__in=urls)]) == set(urls)

# Code I was working out to use Django querysets to pull out relationships among ebooks, editions, works

In [ ]:
from regluit.core.models import (Ebook, Edition, Work)
from django.db.models import (Q, F)

# models.Identifier.objects.filter(edition__isnull=False).filter(~Q(edition__work__id = F('work__id'))).count()

editions_with_ebooks = Edition.objects.filter(ebooks__isnull=False)
editions_with_ebooks

edition = editions_with_ebooks[0]
print (edition.work_id)
work = edition.work
print (work.editions.all())
# didn't know you should use distinct()
Edition.objects.filter(Q(work__id=edition.work_id) & Q(ebooks__isnull=False)).distinct()
#Edition.objects.filter(Q(work__id=edition.work_id))
#work.objects.filter(editions__ebooks__isnull=False)

In [ ]:
# let me grab ebooks and look at their parent works

from regluit.core.models import Ebook

[eb.edition for eb in Ebook.objects.all()]

# Extra

In [ ]:
raise Exception("Stop here")

# Checking Celery Results

In [ ]:
# Checking the results of a local celery task 
from regluit.core import tasks

task_id = "28982485-efc3-44d7-9cf6-439645180d5d"
result = tasks.fac.AsyncResult(task_id)
result.get()